# Prerequisites
Install the `OpenPAI` sdk from `github` and specify your cluster information in `~/.openpai/clusters.yaml`. 

And for simplicity and security, we recommand user to setup necessary information in `.openpai/defaults.json` other than shown in the example notebook. (Refer to for [README](https://github.com/microsoft/pai/blob/sdk-release-v0.4.00/contrib/python-sdk/README.md) more details.)

<font color=red>Please make sure you have set default values for ***cluster-alias***, ***image*** and ***workspace***.</font>
If not, use below commands to set them
```bash
opai set cluster-alias=<your/cluster/alias> workspace=<your/hdfs/workspace>
```

In [ ]:
from openpaisdk.command_line import Engine, get_client

clusters = Engine().process(['cluster', 'list'])
default_values = Engine().process(['set'])

assert ('cluster-alias' in default_values or len(clusters)==1) \
    and 'workspace' in default_values, \
    "Please setup necessary infromation"

# Submit a hello-world job

The `opai` command can help to generate a simplest job config file from user commands and submit it to job. 

In [ ]:
%%writefile hello.sh
echo "Hello, OpenPAI"

In [ ]:
from uuid import uuid4 as randstr
job_name = 'test_cli_sub_' + randstr().hex

! opai job sub -i jupyter/minimal-notebook -j {job_name} -s hello.sh bash code/hello.sh

The generated job_config file will be cached locally for future use.

In [ ]:
from openpaisdk import __jobs_cache__
import os
job_cfg_file = os.path.join(__jobs_cache__, job_name, "job_config.json")
print(job_cfg_file)

# Query job
Use `opai job list` command to query the job status

In [ ]:
Engine().process(['job', 'list', job_name])

In [ ]:
from openpaisdk.command_line import Engine
import time

def wait_for_job(job_name):
    state = Engine().process(['job', 'list', job_name])["jobStatus"].get("state", None)

    while state not in ["SUCCEEDED", "FAILED"]:
        assert state in ["WAITING", "RUNNING"], "uknown status %s" % state
        time.sleep(10)
        state = Engine().process(['job', 'list', job_name])["jobStatus"].get("state", None)

    assert state == "SUCCEEDED", "job failed"
    print("job %s succeeded" % job_name)

wait_for_job(job_name)

# Submit a job from existing config file
Now we show how to submit job from an existing job config file (the one generated in previous cell, the job name would be changed to avoid conflict). 

In [ ]:
job_name_new = 'test_cli_clone_' + randstr().hex

! opai job submit -j {job_name_new} --rename {job_cfg_file}

wait_for_job(job_name_new)